<a href="https://colab.research.google.com/github/KCW-colab/study/blob/main/%EB%AA%A8%EB%91%90%EB%A5%BC%20%EC%9C%84%ED%95%9C%20%EB%94%A5%EB%9F%AC%EB%8B%9D/Lab_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab-05 Logistic Regression

hypothesis

$$ H(X) = 1 / ( 1 + e^{-W^TX}) $$

Cost

$$ cost(W) = -1/m\sum ylog(H(x)) + (1-y)log(1-H(x))$$



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# For reproducibility
torch.manual_seed(1)

In [4]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [5]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [6]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [7]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [8]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [9]:
print('1/(e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(e^{-1}) equals:  tensor([0.7311])


In [10]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [11]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


For one element, compute loss

In [12]:
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

In [13]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [14]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [15]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [16]:
# initailize model

W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# setting optimizer
optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d} / {} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0 / 1000 Cost: 0.693147
Epoch  100 / 1000 Cost: 0.134722
Epoch  200 / 1000 Cost: 0.080643
Epoch  300 / 1000 Cost: 0.057900
Epoch  400 / 1000 Cost: 0.045300
Epoch  500 / 1000 Cost: 0.037261
Epoch  600 / 1000 Cost: 0.031672
Epoch  700 / 1000 Cost: 0.027556
Epoch  800 / 1000 Cost: 0.024394
Epoch  900 / 1000 Cost: 0.021888
Epoch 1000 / 1000 Cost: 0.019852


In [17]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [18]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [19]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [20]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [21]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(8, 1) # W, b 다 들고 있음. self.linear = {W, b}
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

model = BinaryClassifier()

In [24]:
import numpy as np

xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [25]:
print(x_train[0:5])
print(y_train[0:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [26]:
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# setting optimizer 
optimizer = optim.SGD(model.parameters(), lr = 1)


nb_epochs = 100
for epoch in range(nb_epochs + 1):
  hypothesis = model(x_train)

  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {:4d} / {} Cost: {:.6f} Accuracy {:2.2f}'.format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0 / 100 Cost: 0.611617 Accuracy 71.41
Epoch   10 / 100 Cost: 0.523626 Accuracy 75.36
Epoch   20 / 100 Cost: 0.511980 Accuracy 75.76
Epoch   30 / 100 Cost: 0.503711 Accuracy 76.15
Epoch   40 / 100 Cost: 0.497699 Accuracy 76.15
Epoch   50 / 100 Cost: 0.493231 Accuracy 76.42
Epoch   60 / 100 Cost: 0.489824 Accuracy 76.55
Epoch   70 / 100 Cost: 0.487155 Accuracy 76.94
Epoch   80 / 100 Cost: 0.485010 Accuracy 77.08
Epoch   90 / 100 Cost: 0.483251 Accuracy 77.21
Epoch  100 / 100 Cost: 0.481780 Accuracy 77.08
